In [1]:
import sys, os
import pandas as pd
import numpy as np
from statsmodels.tsa.base import tsa_model as tsa
from statsmodels.tsa import holtwinters as hw
sys.path.insert(0, "/home/osboxes/zementis/pmml")
from nyoka.pmml.PMML43Ext import *
from nyoka.pmml.statsmodels import exponential_smoothing as es

In [2]:
def import_data(trend=False, seasonality=False):
    """
    Returns a dataframe with time series values.
    :param trend: boolean
        If True, returns data with trend
    :param seasonality: boolean
        If True, returns data with seasonality
    :return: ts_data: DataFrame
        Index of the data frame is either a time-index or an integer index. First column has time series values
    """
    if trend and seasonality:
        # no of international visitors in Australia
        data = [41.7275, 24.0418, 32.3281, 37.3287, 46.2132, 29.3463, 36.4829, 42.9777, 48.9015, 31.1802, 37.7179,
                40.4202, 51.2069, 31.8872, 40.9783, 43.7725, 55.5586, 33.8509, 42.0764, 45.6423, 59.7668, 35.1919,
                44.3197, 47.9137]
        index = pd.DatetimeIndex(start='2005', end='2010-Q4', freq='QS')
        ts_data = pd.Series(data, index)
        ts_data.index.name = 'datetime_index'
        ts_data.name = 'n_visitors'
        return ts_data
    elif trend:
        # no. of annual passengers of air carriers registered in Australia
        data = [17.5534, 21.86, 23.8866, 26.9293, 26.8885, 28.8314, 30.0751, 30.9535, 30.1857, 31.5797, 32.5776,
                33.4774, 39.0216, 41.3864, 41.5966]
        index = pd.DatetimeIndex(start='1990', end='2005', freq='A')
        ts_data = pd.Series(data, index)
        ts_data.index.name = 'datetime_index'
        ts_data.name = 'n_passengers'
        return ts_data
    elif seasonality:
        pass
    else:
        # Oil production in Saudi Arabia
        data = [446.6565, 454.4733, 455.663, 423.6322, 456.2713, 440.5881, 425.3325, 485.1494, 506.0482, 526.792,
                514.2689, 494.211]
        index = pd.DatetimeIndex(start='1996', end='2008', freq='A')
        ts_data = pd.Series(data, index)
        ts_data.index.name = 'datetime_index'
        ts_data.name = 'oil_production'
        return ts_data

In [3]:
def reconstruct_expon_smooth(pmml_file_name):
    """
    Parses a pmml file and extracts the parameters and time series data. Uses Statsmodels to instantiate a model object
    of exponential smoothing. Returns parameters and model-object
    :param pmml_file_name:
    :return: params: dictionary
        Parameters of the model as key-value pairs
    :return: stsmdl: model object from Statsmodels
        This model object is created using the parameters and time-series data extracted from the pmml file
    """

    def get_ts_data_from_pmml(ts_model_obj):
        time_series_obj = ts_model_obj.get_TimeSeries()[0]
        time_values = time_series_obj.get_TimeValue()
        index = list()
        ts_values = list()
        for time_value in time_values:
            index.append(time_value.get_index())
            ts_values.append(time_value.get_value())
        ts_data = pd.Series(data=ts_values, index=index)
        return ts_data

    def get_params_model_from_pmml(ts_data, ts_model_obj):
        params = dict()
        exp_smooth_obj = ts_model_obj.get_ExponentialSmoothing()
        level_obj = exp_smooth_obj.get_Level()
        trend_obj = exp_smooth_obj.get_Trend_ExpoSmooth()
        season_obj = exp_smooth_obj.get_Seasonality_ExpoSmooth()
        params['smoothing_level'] = level_obj.get_alpha()
        if trend_obj:
            params['smoothing_slope'] = trend_obj.get_gamma()
            params['damping_slope'] = trend_obj.get_phi()
            trend_type = trend_obj.get_trend()
            if trend_type == 'additive':
                trend = 'add'
                damped = False
            elif trend_type == 'multiplicative':
                trend = 'mul'
                damped = False
            elif trend_type == 'damped_additive':
                trend = 'add'
                damped = True
            elif trend_type == 'damped_multiplicative':
                trend = 'mul'
                damped = True
            elif trend_type == 'polynomial_exponential':
                pass
        else:
            trend = None
            damped = False
        if season_obj:
            params['smoothing_seasonal'] = season_obj.get_delta()
            seasonal_periods = season_obj.get_Array().get_n()
            params['initial_seasons'] = np.array(season_obj.get_Array().get_valueOf_().strip().split(' '))
            season_type = season_obj.get_type()
            if season_type == 'additive':
                seasonal = 'add'
            elif season_type == 'multiplicative':
                seasonal = 'mul'
        else:
            seasonal = None
            seasonal_periods = None
        if ts_model_obj.get_Extension():  # if Extension elements exist in pmml file
            for extension in ts_model_obj.get_Extension():
                if extension.get_name() == 'initialLevel':
                    params['initial_level'] = extension.get_value()
                if extension.get_name() == 'initialTrend':
                    params['initial_slope'] = extension.get_value()
        stsmdl = hw.ExponentialSmoothing(ts_data, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods,
                                         damped=damped)
        return params, stsmdl

    nyoka_pmml = parse(pmml_file_name, silence=False)
    ts_model_obj = nyoka_pmml.TimeSeriesModel[0]
    ts_data = get_ts_data_from_pmml(ts_model_obj)
    params, stsmdl = get_params_model_from_pmml(ts_data, ts_model_obj)
    return params, stsmdl

# Simple Exponential Smoothing
# No Trend No Seasonality

In [4]:
ts_data = import_data(trend=False, seasonality=False)

In [5]:
# trend: {add, mul, None}
# seasonal: {add, mul, None}
# damped: {True, False}
# seasonal_periods: {int, None}
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend=None, 
                                    damped=False, 
                                    seasonal=None, 
                                    seasonal_periods=None)
results_obj = model_obj.fit(optimized=True)

In [8]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [9]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [10]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [11]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


# Holt's Exponential Smoothing (only trend, no seasonality)

In [12]:
ts_data = import_data(trend=True, seasonality=False)

** Additive trend. No damping **

In [13]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='add', 
                                    damped=False, 
                                    seasonal=None, 
                                    seasonal_periods=None)
results_obj = model_obj.fit(optimized=True)

In [14]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [15]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [16]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [17]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


** Addditve trend with damping**

In [18]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='add', 
                                    damped=True, 
                                    seasonal=None, 
                                    seasonal_periods=None)
results_obj = model_obj.fit(optimized=True)

In [19]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [20]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [21]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [22]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


** Multiplicative trend. No damping**

In [23]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='mul', 
                                    damped=False, 
                                    seasonal=None, 
                                    seasonal_periods=None)
results_obj = model_obj.fit(optimized=True)

In [24]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [25]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [26]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [27]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


** Multiplicative trend with damping**

In [28]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='mul', 
                                    damped=True, 
                                    seasonal=None, 
                                    seasonal_periods=None)
results_obj = model_obj.fit(optimized=True)

In [29]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [30]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [31]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [32]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


# Holt - Winter Exponential Smoothing (trend and seasonality present)

** Additive trend, additive seasonality, no damping **

In [33]:
ts_data = import_data(trend=True, seasonality=True)

In [34]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='add', 
                                    damped=False, 
                                    seasonal='add', 
                                    seasonal_periods=4)
results_obj = model_obj.fit(optimized=True)

In [35]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [36]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [37]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [38]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


** Additive trend, additive seasonality, with damping **

In [39]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='add', 
                                    damped=True, 
                                    seasonal='add', 
                                    seasonal_periods=4)
results_obj = model_obj.fit(optimized=True)

In [40]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [41]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [42]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [43]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


** Additive trend, multiplicative seasonality, no damping **

In [45]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='add', 
                                    damped=False, 
                                    seasonal='mul', 
                                    seasonal_periods=4)
results_obj = model_obj.fit(optimized=True)

In [47]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [48]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [49]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [50]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful


** Additive trend, multiplicative seasonality with damping **

In [52]:
model_obj = hw.ExponentialSmoothing(ts_data, 
                                    trend='add', 
                                    damped=False, 
                                    seasonal='mul', 
                                    seasonal_periods=4)
results_obj = model_obj.fit(optimized=True)

In [53]:
pmml_file_name = 'exponential_smoothing.pmml'
es.ExponentialSmoothingToPMML(ts_data, model_obj, results_obj, pmml_file_name);

In [54]:
%%capture
params_recon, model_obj_recon = reconstruct_expon_smooth(pmml_file_name);

In [55]:
pred_val = model_obj.predict(results_obj.params)
pred_val_recon = model_obj_recon.predict(params_recon)

In [56]:
if np.allclose(pred_val, pred_val_recon):
    print('reconstruction successful')
else:
    print('reconstruction error')

reconstruction successful
